In [125]:
import math
import numpy as np
import matplotlib.pyplot as plt

## American call and put options pricing by following the algorithm:##

Step1:

Create final spot values using the formula

$$ So * e^{(r-d- \frac{1}{2}\sigma^2)T+ \sigma \sqrt{\frac{T}{N}} i} $$
where i belongs to {-N,..., N} 


Step2:

Evaluate payoff and store the value (s_up, s_down)

In [126]:
def american_option(S0, sigma, r, d, K, option, N, T):
    s_up=[] # for up spots
    s_down=[] #for down spots
    T=T/365
    next_up=[]
    next_down=[]
    for i in range(1, N+1):
        a= (r - d -(1/2)*math.pow(sigma,2))*T
        b_up= sigma*math.sqrt(T/N)*i
        b_down= sigma*math.sqrt(T/N)*(-i)
        if (option.lower()=='call'):
            #calculate payoff function for call 
            s_up.append(max((S0 * math.exp(a+b_up))-K,0))
            s_down.append(max((S0 * math.exp(a+b_down))-K,0))
        if (option.lower()=='put'):
            #calculate payoff function for put
            s_up.append(max(K-(S0 * math.exp(a+b_up)),0))
            s_down.append(max(K-(S0 * math.exp(a+b_down)),0))
    print('\n final up spot values: ')
    print(s_up)
    print('\n final down spot values: ')
    print(s_down)
    
    
    next_up=s_up
    next_down=s_down
    for t in range(N-1,0,-1): #going backwords from period N-1 to 1 stoping when we reach zero
        #to calculate at previous time as we go backwords
        next_up, next_down= calculate_Pspot(S0, sigma, r, d, K, option, N, T, t, next_up, next_down)
        
    return(next_up, next_down) #returning the last two values of period 1


Step3:

At previous time compute posible spot values using formula
$$ So * e^{(r-d- \frac{1}{2}\sigma^2)(N-1)\frac{T}{N}+ \sigma \sqrt{\frac{T}{N}} i} $$
where i belongs to {-(N-1),..., (N-1)} 


In [143]:
#to calculate at previous time
def calculate_Pspot(S0, sigma, r, d, K, option, N, T, t, next_up, next_down):
    previous_up=[]
    previous_down=[]
    E=math.exp(-r*(T/N)) #calculate e^-rt (discount)
    for i in range(1, t+1):
        a= (r - d -(1/2)*math.pow(sigma,2))*(N-1)*T/N
        b_up= sigma*math.sqrt(T/N)*i
        b_down= sigma*math.sqrt(T/N)*(-i)
        if (option.lower()=='call'):
            #calculate payoff function for each up spot
            previous_up.append(max((S0 * math.exp(a+b_up))-K,0))
            previous_down.append(max((S0 * math.exp(a+b_down))-K,0))
        if (option.lower()=='put'):
            #calculate payoff function for each down spot
            previous_up.append(max(K-(S0 * math.exp(a+b_up)),0))
            previous_down.append(max(K-(S0 * math.exp(a+b_down)),0))
        
    #to take max with the discounted payoff and two possible values of next time
    next_up,next_down=compare(previous_up, previous_down, next_up, next_down,E)
    
    print('\n %dth up spot values: '%t)
    print(next_up)
    print('\n %dth down spot values: '%t)
    print(next_down)
    return(next_up,next_down)

Step4:

Take max with the discounted payoff of the  two possible values of next time

In [144]:
def compare(previous_up, previous_down, next_up, next_down,E):
    for i in range(0, len(previous_up)):
        previous_up[i]= max(previous_up[i], E*next_up[i], E*next_down[i])
    for j in range(0, len(previous_down)):
        previous_down[j]= max(previous_down[j], E*next_up[j+1], E*next_down[j+1])
    
    return(previous_up,previous_down)

In [145]:
american_option(100, 0.3, 0.1, 0.05, 90,   "call", 12, 365)


 final up spot values: 
[19.592914792072406, 29.50703820437974, 40.31802473252759, 52.10700746464718, 64.9624590459031, 78.98085563815101, 94.26740094349682, 110.93681572532807, 129.11419875192735, 148.93596562378158, 170.5508725301936, 194.12113261817115]

 final down spot values: 
[2.1638199878621833, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

 11th up spot values: 
[19.54726058952842, 29.457253977562104, 40.263736866313224, 52.04780854550154, 64.89790480436824, 78.91046161474176, 94.19063885298311, 110.85310949211882, 129.0229201868024, 148.8364297095272, 170.44233228076166]

 11th down spot values: 
[29.262167929228085, 39.98343724389378, 51.67458665823417, 64.42335460882215, 78.32541663007882, 93.485103369373, 110.01618355604863, 128.04271780031908, 147.6999896295234, 169.13552074888116, 192.51017814596298]

 10th up spot values: 
[29.01932976083135, 39.65162639891484, 51.24575438555044, 63.888724041701614, 77.67541691852767, 92.70929785388614, 109.10319144372927, 126.98012875557124, 146.

([155.61219118043016], [177.11791416248926])

In [146]:
american_option(100, 0.3, 0.1, 0.05, 90,   "put", 12, 365)


 final up spot values: 
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

 final down spot values: 
[0, 5.481951334381634, 12.493441014222341, 18.9232680988343, 24.819686438702533, 30.22694681774781, 35.18562904168347, 39.73294647682369, 43.90302532481211, 47.72716072914071, 51.234051635458655, 54.45001616822466]

 11th up spot values: 
[0, 5.4364582244896775, 12.389761603311845, 18.76622943456598, 24.613715124152883, 29.97610223168706, 34.89363380294947, 39.40321438141733, 43.53868696537348, 47.33108698907223, 50.80887523401127]

 11th down spot values: 
[5.4364582244896775, 12.389761603311845, 18.76622943456598, 24.613715124152883, 29.97610223168706, 34.89363380294947, 39.40321438141733, 43.53868696537348, 47.33108698907223, 50.80887523401127, 53.99815141823609]

 10th up spot values: 
[5.391342648603661, 12.28694259749187, 18.610493988216803, 24.409453105267467, 29.727339331439154, 34.60406174725257, 39.07621863617831, 43.17737214336026, 46.938300149066016, 50.38722724709635]

 10th down spot 

([46.74642187265022], [49.680697612656594])